<a href="https://colab.research.google.com/github/rkrissada/100DayOfMLCode/blob/master/day_044_Improvement_CNN_on_MNIST_by_datagen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.datasets import mnist

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 2s 0us/step


In [0]:
from keras import backend as k 

img_rows, img_cols=28, 28

if k.image_data_format() == 'channels_first': 
  x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols) 
  x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols) 
  inpx = (1, img_rows, img_cols) 

else: 
  x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1) 
  x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1) 
  inpx = (img_rows, img_cols, 1) 

x_train = x_train.astype('float32') 
x_test = x_test.astype('float32') 
x_train /= 255
x_test /= 255

In [0]:
import keras  

y_train = keras.utils.to_categorical(y_train) 
y_test = keras.utils.to_categorical(y_test) 

In [0]:
datagen = ImageDataGenerator(
        rotation_range=15,
        zoom_range = 0.15,  
        width_shift_range=0.1, 
        height_shift_range=0.1)

In [8]:
nets = 15
model = [0] *nets
for j in range(nets):
    model[j] = Sequential()

    model[j].add(Conv2D(32, kernel_size = 3, activation='relu', input_shape = (28, 28, 1)))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(32, kernel_size = 3, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Dropout(0.4))

    model[j].add(Conv2D(64, kernel_size = 3, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(64, kernel_size = 3, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Dropout(0.4))

    model[j].add(Conv2D(128, kernel_size = 4, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Flatten())
    model[j].add(Dropout(0.4))
    model[j].add(Dense(10, activation='softmax'))

    # COMPILE WITH ADAM OPTIMIZER AND CROSS ENTROPY COST
    model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
# DECREASE LEARNING RATE EACH EPOCH
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)
# TRAIN NETWORKS
history = [0] * nets
epochs = 45
for j in range(nets):
    X_train2, X_val2, Y_train2, Y_val2 = train_test_split(x_train, y_train, test_size = 0.1)
    history[j] = model[j].fit_generator(datagen.flow(X_train2,Y_train2, batch_size=64),
        epochs = epochs, steps_per_epoch = X_train2.shape[0]//64,  
        validation_data = (X_val2,Y_val2), callbacks=[annealer], verbose=0)
    print("CNN {0:d}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
        j+1,epochs,max(history[j].history['acc']),max(history[j].history['val_acc']) ))

CNN 1: Epochs=1, Train accuracy=0.99290, Validation accuracy=0.99817
CNN 2: Epochs=1, Train accuracy=0.98758, Validation accuracy=0.99550
CNN 3: Epochs=1, Train accuracy=0.98799, Validation accuracy=0.99667
CNN 4: Epochs=1, Train accuracy=0.98766, Validation accuracy=0.99500
CNN 5: Epochs=1, Train accuracy=0.98753, Validation accuracy=0.99450
CNN 6: Epochs=1, Train accuracy=0.98751, Validation accuracy=0.99567
CNN 7: Epochs=1, Train accuracy=0.98732, Validation accuracy=0.99267
CNN 8: Epochs=1, Train accuracy=0.98819, Validation accuracy=0.99483
CNN 9: Epochs=1, Train accuracy=0.98884, Validation accuracy=0.99467
CNN 10: Epochs=1, Train accuracy=0.98020, Validation accuracy=0.99267
CNN 11: Epochs=1, Train accuracy=0.86864, Validation accuracy=0.97700
CNN 12: Epochs=1, Train accuracy=0.87823, Validation accuracy=0.98200
CNN 13: Epochs=1, Train accuracy=0.87643, Validation accuracy=0.97717
CNN 14: Epochs=1, Train accuracy=0.87085, Validation accuracy=0.98433
CNN 15: Epochs=1, Train accur

In [0]:
# ENSEMBLE PREDICTIONS AND SUBMIT
results = np.zeros( (x_test.shape[0],10) ) 
for j in range(nets):
    results = results + model[j].predict(x_test)
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.to_csv("MNIST-CNN-ENSEMBLE.csv",index=False)